In [1]:
import open3d
import torch
import pytorch3d
from pytorch3d.io import load_obj
from scipy.spatial.transform import Rotation as Rotation

from pytorch3d.renderer.cameras import PerspectiveCameras, OrthographicCameras

#Load meshes and visualize it with Open3D
mesh_file = "cube.obj"
print('visualizing the mesh using open3D')
mesh = open3d.io.read_triangle_mesh(mesh_file)
'''open3d.visualization.draw_geometries([mesh],
                                     mesh_show_wireframe = True,
                                     mesh_show_back_face = True,
                                     )'''

#Define a mini-batch of 8 cameras
image_size = torch.ones(8, 2)
image_size[:,0] = image_size[:,0] * 1024
image_size[:,1] = image_size[:,1] * 512
image_size = image_size.cuda()

focal_length = torch.ones(8, 2)
focal_length[:,0] = focal_length[:,0] * 1200
focal_length[:,1] = focal_length[:,1] * 300
focal_length = focal_length.cuda()

principal_point = torch.ones(8, 2)
principal_point[:,0] = principal_point[:,0] * 512
principal_point[:,1] = principal_point[:,1] * 256
principal_point = principal_point.cuda()

R = Rotation.from_euler('zyx', [
[n*5, n, n]  for n in range(-4, 4, 1)], degrees=True).as_matrix()
R = torch.from_numpy(R).cuda()
T = [ [n, 0, 0] for n in range(-4, 4, 1)]
T = torch.FloatTensor(T).cuda()

#print('R = ', R)
#exit()

camera = PerspectiveCameras(focal_length = focal_length,
                            principal_point = principal_point,
                            in_ndc = False,
                            image_size = image_size,
                            R = R,
                            T = T,
                            device = 'cuda')

world_to_view_transform = camera.get_world_to_view_transform()
world_to_screen_transform = camera.get_full_projection_transform()
print('The type of world_to_screen_transform = ', type(world_to_screen_transform))

#Load meshes using PyTorch3D
vertices, faces, aux = load_obj(mesh_file)
vertices = vertices.cuda()

world_to_view_vertices = world_to_view_transform.transform_points(vertices)
world_to_screen_vertices = world_to_screen_transform.transform_points(vertices)

print('world_to_view_vertices = ', world_to_view_vertices)
print('world_to_screen_vertices = ', world_to_screen_vertices)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
visualizing the mesh using open3D
The type of world_to_screen_transform =  <class 'pytorch3d.transforms.transform3d.Transform3d'>
world_to_view_vertices =  tensor([[[-32.2545, -64.8477,  19.9112],
         [-33.1470, -64.4302,   9.9598],
         [-66.8084,  29.4765,  16.9185],
         [-65.9159,  29.0590,  26.8698],
         [ 61.4859, -30.7290,  12.9355],
         [ 60.5934, -30.3115,   2.9842],
         [ 26.9319,  63.5952,   9.9428],
         [ 27.8244,  63.1777,  19.8942]],

        [[-37.1586, -61.9293,  19.9488],
         [-37.7989, -61.5590,   9.9762],
         [-63.3807,  34.9721,  15.2026],
         [-62.7405,  34.6018,  25.1752],
         [ 59.3016, -36.0828,  14.7152],
         [ 58.6613, -35.7126,   4.7426],
         [ 33.0795,  60.8185,   9.9690],
         [ 33.7197,  60.4483,  19.9416]],

        [[-41.7